<a href="https://colab.research.google.com/github/harshitadd/CrossSiloFLDemo/blob/master/PreVID_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
##Importing Mini Batch Data 
import os 
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install pydicom 
import pydicom 
import cv2
import matplotlib.pyplot as plt 

     |████████████████████████████████| 35.3MB 94kB/s 


In [0]:
dcm_path=os.listdir('/content/drive/My Drive/Fed_Covid/minibatch/')
dcm_data={}

alpha = 1.5 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)

ctr=0
for file in dcm_path:
  name = '/content/drive/My Drive/Fed_Covid/minibatch/' + file
  temp = pydicom.dcmread(name)
  image = temp.pixel_array
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  adjusted = cv2.resize(image,(64,64))
  dcm_data[file]=adjusted
  ctr+=1
  print(ctr)

In [0]:
import csv 
labels=[]
with open('/content/drive/My Drive/Fed_Covid/stage_2_train_labels.csv','r') as file:
  reader = csv.reader(file)
  for row in reader:
    labels.append(row)

In [0]:
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
scaler = MinMaxScaler()
pid=[]
dicom=[]
label=[]
cid = 0
for PID in labels:
  for key in dcm_data:
    if(key[:-4]==PID[0]):
      l=[]
      for val in dcm_data[key]:
        l.append(scaler.fit_transform(val))
      l = np.reshape(l,(3,64,64))
      dicom.append(l)
      label.append(int(PID[5]))
      

In [13]:
import numpy as np
print(np.shape(dicom[0]))

(64, 64, 3)


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
from torch.utils.data import DataLoader, Dataset

In [0]:
import pandas as pd 
df = pd.DataFrame()
df['features']=dicom
df['labels']=label

In [19]:
df

,features,labels
0,"[[[0.0, 0.0, 0.0, 0.005405405405405406, 0.0054...",0
1,"[[[1.0, 1.0, 1.0, 0.37305699481865284, 0.37305...",0
2,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0
3,"[[[0.005208333333333333, 0.005208333333333333,...",0
4,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0
...,...,...
424,"[[[0.013333333333333332, 0.013333333333333332,...",1
425,"[[[0.018404907975460124, 0.018404907975460124,...",0
426,"[[[0.0, 0.0, 0.0, 0.2066115702479339, 0.206611...",0
427,"[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0...",0


In [0]:
class data_maker(Dataset):
  def __init__(self, data):
        self.data = data
        
  def __len__(self):
      return len(self.data)
  
  def __getitem__(self, index):
      target = np.array(self.data['labels'][index])
      data_val = np.array(self.data['features'][index])

      return data_val,target
      
        

In [0]:
train_data = data_maker(df)

In [0]:
def get_train_data(batch_size):
      train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=False)
      return train_loader

In [26]:
train_loader= get_train_data(batch_size=8)
n_batches = len(train_loader)
for i, data in enumerate(train_loader, 0):
       inputs, labels = data
       print(np.shape(inputs))
       break   

torch.Size([8, 3, 64, 64])


In [0]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Input - (32,1024,1024)
  
        self.conv1 = nn.Conv2d(3,32, kernel_size=8, stride = 2)  #I/P - 1*32
        self.pool1 = nn.MaxPool2d(kernel_size=4,stride=2,padding=0)  
        self.conv2 = nn.Conv2d(32, 64, 8)
        self.pool2 = nn.MaxPool2d(4,2,padding=0)
        self.fc1 = nn.Linear(256, 64*64)
        self.fc2 = nn.Linear(64*64, 64)
        self.fc3 = nn.Linear(64, 8)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        # Input of 1*1024*1024 going to 32*1024*1024  - 1*64*64 going to 32*64*64
        x = self.pool1(x)
        # Input of 32*1024*1024 going to 32*256*256 - 32*64*64 going to 32*16*16
        x = F.relu(self.conv2(x))
        # Input of 32*256*256 to 64*256*256 - 32*16*16 to 64*16*16
        x = self.pool2(x)
        # Input of 64*256*256 to 64*64*64 - 64*16*16 going to 64*4*4

        x = x.view(-1,64*4)
     
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)

In [0]:
def createLossAndOptimizer(net, learning_rate=0.001):
    
    #Loss function
    loss = torch.nn.CrossEntropyLoss()
    
    #Optimizer
    optimizer = optim.Adam(net.parameters(), lr=learning_rate)
    
    return(loss, optimizer)

In [0]:
import time
from torch.autograd import Variable
def trainNet(net, batch_size, n_epochs, learning_rate):
    
    
    #Get training data
    train_loader = get_train_data(batch_size=8)
    n_batches = len(train_loader)
    
    #Create our loss and optimizer functions
    loss, optimizer = createLossAndOptimizer(net, learning_rate)
    
    #Time for printing
    training_start_time = time.time()
    
    #Loop for n_epochs
    for epoch in range(n_epochs):
        
        running_loss = 0.0
        print_every = n_batches // 10
        start_time = time.time()
        total_train_loss = 0
        
        for i, data in enumerate(train_loader, 0):
            
            #Get inputs
            inputs, labels = data
      
            #Wrap them in a Variable object
            inputs, labels = Variable(inputs), Variable(labels)
            
            #Set the parameter gradients to zero
            optimizer.zero_grad()
            
            #Forward pass, backward pass, optimize
            outputs = net(inputs)
            loss_size = loss(outputs, labels)
            loss_size.backward()
            optimizer.step()
            
            #Print statistics
            running_loss += loss_size.data
            total_train_loss += loss_size.data
            
            #Print every 10th batch of an epoch
            if (i + 1) % (print_every + 1) == 0:
                print("Epoch {}, {:d}% \t train_loss: {:.2f} took: {:.2f}s".format(
                        epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every, time.time() - start_time))
                #Reset running loss and time
                running_loss = 0.0
                start_time = time.time()
            
        #At the end of the epoch, do a pass on the validation set
        total_val_loss = 0
        
    print("Training finished, took {:.2f}s".format(time.time() - training_start_time))

In [56]:
CNN = Net()
trainNet(CNN.double(), batch_size=8, n_epochs=2
         , learning_rate=0.001)

Epoch 1, 11% 	 train_loss: 1.43 took: 0.45s
Epoch 1, 22% 	 train_loss: 0.77 took: 0.45s
Epoch 1, 33% 	 train_loss: 0.55 took: 0.45s
Epoch 1, 44% 	 train_loss: 1.09 took: 0.45s
Epoch 1, 55% 	 train_loss: 0.92 took: 0.46s
Epoch 1, 66% 	 train_loss: 0.50 took: 0.47s
Epoch 1, 77% 	 train_loss: 0.64 took: 0.44s
Epoch 1, 88% 	 train_loss: 0.38 took: 0.45s
Epoch 1, 100% 	 train_loss: 0.70 took: 0.43s
Epoch 2, 11% 	 train_loss: 0.67 took: 0.46s
Epoch 2, 22% 	 train_loss: 0.71 took: 0.46s
Epoch 2, 33% 	 train_loss: 0.46 took: 0.46s
Epoch 2, 44% 	 train_loss: 0.92 took: 0.45s
Epoch 2, 55% 	 train_loss: 0.82 took: 0.45s
Epoch 2, 66% 	 train_loss: 0.48 took: 0.45s
Epoch 2, 77% 	 train_loss: 0.74 took: 0.44s
Epoch 2, 88% 	 train_loss: 0.38 took: 0.45s
Epoch 2, 100% 	 train_loss: 0.55 took: 0.44s
Training finished, took 8.11s


https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html - Test Loader

PYTORCH IMPLEMENTATION


In [0]:
!pip install syft

In [0]:
import syft as sy 

hook = sy.TorchHook(torch)

In [0]:
hospital = sy.VirtualWorker(hook, id="hospital")  
clinic = sy.VirtualWorker(hook, id="clinic")  

In [0]:
client_datasets=[(data_hospital,data_hospital_target),(data_clinic, data_clinic_target)]